# Подготовка данных про инфекционные заболевания

Данные к онлайн-приложению https://epistat.wiv-isp.be

Каждая строчка в наборе данных - это запись о заболевании конкретного человека. Т.е. по одной записи можно установить, когда и с кем случилась неприятность. В некоторых случаях это серьезные заболевания вроде СПИД.

Однако, при анализе нас интересует количество случаев заболеваний за период времени в том или ином районе среде той или иной части населения.
Чтобы получать такие числа мы должны правильно отбирать строки в таблице и просто считать количество строк в полученных выборках.

Эпидемиология - наука на стыке микробиологии и социологии.




In [ ]:
%pylab inline
from ipywidgets import interact
import pandas as pd
import seaborn as sns

import scipy.signal as signal
import scipy.stats as stats

Загружаем данные.

In [ ]:
D = pd.read_csv("epistat.csv")  #https://epistat.wiv-isp.be/data/public_cases.csv")
D.info()

Для удобства сделаем колонки для отбора по годам и по месяцам.

In [ ]:
D['Y'] = D.DateMonday.str.slice(0,4).astype(int)
D.Y.value_counts()

In [ ]:
D['YM'] = D.DateMonday.str.slice(0,7)

D['M'] = D.DateMonday.str.slice(5,7).astype(int)
D.M.value_counts().sort_index()

Можно создать новые колонки с латинскими названиями возбудителей и названиями провинций.

In [ ]:
subj_species = {
    'BORPER':'Bordetella pertussis',
    'BRRBUR':'Borrelia burgdorferi',
    'CAM_SP':'Campylobacter sp.',
    'CHIK  ':'Chikungunya virus',
    'CHLPSI':'Chlamydia psittaci',
    'CHLTRA':'Chlamydia trachomatis',
    'CRS_SP':'Cryptosporidium',
    'ENTHIS':'Entamoeba histolytica',
    'GIA_SP':'Giardia sp.',
    'HAEINF':'Haemophilus influenzae',
    'HIV   ':'Lentivirus "HIV"',
    'LIS_SP':'Listeria sp.',
    'MENI  ':'Neisseria meningitidis',
    'NEIGON':'Neisseria gonorrhoeae',
    'PNEU  ':'Streptococcus pneumoniae',
    'SALM  ':'Salmonella',
    'SHIG  ':'Shigella',
    'STRPYO':'Streptococcus pyogenes',
    'VTEC  ':'Escherichia coli "VTEC"',
    'V_ADV ':'Adenovirus',
    'V_HAV ':'Hepatitis A virus',
    'V_HCV ':'Hepatitis C virus',
    'V_HTV ':'Hantavirus',
    'V_PIV ':'Parainfluenza',
    'V_RSV ':'Respiratory Syncytial Virus',
    'V_RTV ':'Rotavirus',
    'YERENT':'Yersinia enterocolitica',
    'ZIKA  ':'Flavivirus "ZIKA"',
}
subj_species = {k.strip():v for k,v in subj_species.items()}
D['Species'] = D.Subject.apply(lambda code: subj_species[code])

In [ ]:
NUTS2_Province = {
10: 'Brussels',
21: 'Antwerp',
22: 'Limburg',
23: 'East Flanders',
24: 'Flemish Brabant',
25: 'West Flanders',
31: 'Walloon Brabant',
32: 'Hainaut',
33: 'Liege',
34: 'Luxembourg',
35: 'Namur',
0: '-'
}
D.NUTS2 = D.NUTS2.fillna(0).astype(int)
D['Province'] = D.NUTS2.apply(lambda code: NUTS2_Province[code])

## Анализ пустот

Перед тем как делать обобщающие выводы по результатам решения задач, прикинем полноту представленных в наборе записей.

Чувствительные данные, относящиеся к самим пациентам, могут отсутствовать умышленно из соображений анонимности при сборе данных. Колонки возраст, пол и провинция могут косвенно указывать на носителей заболевания.

Посмотрим, для каких возбудителей чаще всего не указана провинция.

In [ ]:
D[D['Province']=='-'].Species.value_counts()

Интересно, много случаев СПИД без указания провинции. Еще больше сальмонеллёза. Давайте рассчитаем процент "скрытых" адресов относительно общей численности записей по каждой инфекции.

In [ ]:
(D[D['Province']=='-'].Species.value_counts() / D.Species.value_counts()).sort_values() * 100

Действительно, чаще всего фиксировались без указания адреса случаи заражения ВИЧ (HIV) - 26% от всей статистики.

А что с возрастом инфицированных ВИЧ в Бельгии?

In [ ]:
X = D[(D.Subject=='HIV') & (D.Age.notnull())]

sns.kdeplot(X[X.Province == '-'].Age, label='Без адреса')
sns.kdeplot(X[X.Province != '-'].Age, label='Провинция указана');

Большинство случаев заражения ВИЧ в детском возрасте (до 15 лет) в базе без указания адреса. В остальном распределения примерно совпадают.

А при каких заболеваниях чаще не указывали возраст?

In [ ]:
(D[D.Age.isnull()].Species.value_counts() / D.Species.value_counts()).sort_values() * 100

Сальмонелла - кишечная инфекция. Ничего особенного.

## Динамика

При рассмотрении динамики заболеваемости во времени важно четко представлять иерархичность циклов, которыми оперируют эпидемиологи (как и другие люди).

Глобальные тренды оценивают по годам. 
Внутри года смотрят помесячную динамику.
В данном наборе минимальный срок - неделя, которая имеет четкую длительность - 7 дней.

Проблема осложняется тем, что в месяце нет четкого количества недель, потому что количество дней в разных месяцах варьирует.
Для правильного пересчета значений в иные периодические шкалы нужно использовать календарь. Функционал пересчета дат и периодов из одной шкалы в другую поддерживается в библиотеке `pandas`.

In [ ]:
# создадим новую колонку, которая будет содержать объекты типа `datetime64`, со значениями, 
# дублирующими колонку DateMonday, которая содержит строки из 10 символов.
D['W'] = pd.to_datetime(D['DateMonday'])
# Чтобы легко было суммировать случаи заболевания, добавим в каждую строку по единице.
D['n']= 1

# Теперь полученную колонку сделаем индексом
D.set_index('W', drop=False, inplace=True)
# Индекс преобразуем в период - вместо одного дня понедельника будет диапазон из 7 дней
D = D.to_period('W', copy=False)
D.index.name = 'Time'

При трактовке динамики полезно понимать к какому типу инфекции относится то или иное заболевание. 

Эпидемиологические вспышки напрямую зависят от способа заражения. А он принципиально разный для:
- простудно-легочных заболеваний (Bordetella, Streptococcus, Haemophilus)
- кишечных расстройств (Shigella, Salmonella, Giardia)
- ЗППП (Neisseria gonorrhoeae, Chlamydia trachomatis)
- в общем безвредных паразитов, которые в редких случаях переходят в тяжелые системные заболевания (дизентерийная амёба, ВИЧ, Borrelia, Listeria, Neisseria meningitidis)

Предсказуемость динамики заболеваемости осложняется тем, что безобидный микроорганизм, который легко распознается и ликвидируется иммунными клетками, в отдельных случаях приводит к заболеванию.

Самый яркий пример - детские инфекции. Несмотря на то, что у новорожденных сильный иммунитет, часть которого приобретена пассивно через плаценту, при встрече с некоторыми микропаразитами врожденный иммунитет бессилен. А значит каждому ребенку "надо переболеть" ротавирусной (понос) или аденовирусной (сопли) инфекциями.

### Сезонная динамика

Восприимчивость к патогенам зависит от погоды: зимой легче заболевают одними инфекционными болезнями, летом - другими.

In [ ]:
X = D[D.Species=='Rotavirus']

Для отображения динамики мы должны сгруппировать записи по неделям, сосчитать количество случаев в каждой группе и нарисовать график.

In [ ]:
X.groupby('W').n.count().plot();

Так в колонке `n` одни единички, можно вместо подсчета использовать сумму. График будет тот же самый.

In [ ]:
X.groupby('W').n.sum().plot();

Для наложения графиков годовой динамики один поверх другого нужно сделать то же самое, но для выборки по каждому году. При итерации групп в цикле мы на каждом шаге получаем имя группы и выборку строк для этой группы.

In [ ]:
for glabel,g in X.groupby('Y'):
    # группируем по неделям и берем суммарные значения
    vv = g.groupby('W').n.sum().values    
    plot(vv, label=glabel)

xlabel('Номер недели')
ylabel('Заболеваний / нед')
legend();

Четко виден резкий рост заболеваемости на 5-10 неделях года.

Для более гладкой динамики можно сгруппировать по месяцам.

In [ ]:
for glabel,g in X.groupby('Y'):
    # группируем по неделям и берем суммарные значения
    vv = g.groupby('M').n.sum()#.values
    
    plot(vv, label=glabel)

xlabel('Месяц')
ylabel('Заболеваний / мес')
legend();

Напомним возрастной состав заболевших ротавирусной инфекцией.

In [ ]:
X.Age.hist(bins=50);

Маленькие дети и несколько глубоких старичков.

Рассмотрим детальней каждую категорию с учетом пола. При наложении нескольких распределений удобно строить ядерные оценки плотности (KDE, Kernel Density Estimation).

In [ ]:
X_ = X[X.Age>60]
sns.kdeplot(X_[X_.Gender == 'M'].Age, label='M', color='cyan');
sns.kdeplot(X_[X_.Gender == 'F'].Age, label='F', color='coral');

Если строить классические гистограммы, то полупрозрачность дает возможность легко обнаруживать преобладание той или другой группы.

Так как возраст указан с точностью до года, то для подбробной гистограммы лучше задать не количество классов, а список границ классов, тогда каждому году будет четко соответствовать один столбик. 

In [ ]:
X_ = X[X.Age<20]
classes = arange(16)
hist(X_[X_.Gender == 'M'].Age, label='M', color='cyan', alpha=.5, bins=classes);
hist(X_[X_.Gender == 'F'].Age, label='F', color='coral',alpha=.5, bins=classes);
xticks(classes+.5, classes)
legend();

А старички также болеют весной?

Наложим нормализованную динамику для двух возрастных групп.

In [ ]:
X_ = X[X.Age>60]
(X_.groupby('W').n.sum() / X_.n.sum()).plot(label='старички');
X_ = X[X.Age<8]
(X_.groupby('W').n.sum() / X_.n.sum()).plot(label='груднички');
legend();

Да, тоже по весне, но не так регулярно из-за малого количества случаев.

### Годовая динамика

Почему же все-таки в 2014 году была аномалия в заболеваемости ротавирусной инфекцией?
Почему детки из северной Бельгии в марте 2014 болели мало, а переболели только в следующем 2015 году?

Чтобы ответить на этот вопрос удобно рассмотреть изменения между годами независимо от изменений внутри года. Расположим оси смены лет и смены месяцев перпендикулярно, как это мы уже делали для визуализации температурных изменений.

In [ ]:
momo = [datetime.date.strftime(x,'%b') for x in pd.DatetimeIndex(start='2000-01', end='2001-01', freq='M')]

N = X.groupby(['Y','M']).n.count().unstack()
N.columns=momo
sns.heatmap(N, cmap='Blues');

Группировка по двум осям является типичной операцией, поэтому есть специальный метод - на русский переводится как "сводная таблица".

In [ ]:
N = X.pivot_table('n', 'Y', 'M', 'sum')
N.columns=momo
sns.heatmap(N, cmap='Blues');

Давайте посмотрим отдельно для грудничков и годовалых младенцев.

In [ ]:
fig, ax = subplots(1,2, sharey=True, figsize=(12,4))

for iage, age in enumerate([0,1]):
    sca(ax[iage])
    N = X[X.Age==age].pivot_table('n', 'Y', 'M', 'sum')
    N.columns=momo
    sns.heatmap(N, cmap='Blues');
    title('Age = {}'.format(age))


Паттерн не различался. Значит дело не в возрасте.

Посмотрим паттерн температур для [Западной Фландрии](//ru.wikipedia.org/wiki/Западная_Фландрия) с административным центром — городом Брюгге.

По координатам 51°00′ с. ш. 3°00′ в. д. можно найти данные метеорологических станций например с помощью сервиса http://climexp.knmi.nl/selectstation.cgi

Ближайшая станция в [Лилле, Франция](
http://climexp.knmi.nl/gettempall.cgi?id=someone@somewhere&WMO=7015&STATION=LILLE&extraargs=)



На исходных данных видно отсутствие понижения среднемесячной температуры зимой 2014 года ниже 5 градусов по Цельсию

![](http://climexp.knmi.nl/data/ta7015_2008:2018.png)

На рисунке показаны температурные аномалии, т.е. отклонения в градусах от ожидаемой температуры.

![](http://climexp.knmi.nl/data/ta7015_2008:2018_a.png)



Действительно в начале 2014 года температура держалась выше, чем обычно на 2 градуса.

Отобразим эти данных в виде `heatmap`.

In [ ]:
u = 'http://climexp.knmi.nl/data/ta7015.dat'
T = pd.read_table(u, delim_whitespace=True, comment='#', header=None, na_values=-999.9,
                 names=momo)
T.tail()

In [ ]:
fig, ax = subplots(1,2, sharey=True, figsize=(12,4))

sca(ax[0])
sns.heatmap(T.loc[2008:2015], cmap='RdBu_r', center=8)
title('Температура');


sca(ax[1])
N = X[X.Province.isin(['West Flanders','East Flanders'])].pivot_table('n', 'Y', 'M', 'sum')
N.columns=momo
sns.heatmap(N, cmap='Blues');
title('Заболеваемость ротавирусной инфекцией во Фландрии');


В 2013 году была относительно холодная зима и март, что сопровождалось всплеском заболеваемости в марте и апреле. В 2014 году было необычно тепло и весеннее обострение прошло без мартовского пика.

Дабы выразить связь в численном виде посчитаем корреляцию между рядами для марта.

In [ ]:
N['Mar'].plot(legend=False)
ylabel('Заболеваемость')
T.loc[2008:2015,'Mar'].plot(label='T', secondary_y=True);
ylabel('Температура')
legend();

In [ ]:
corrcoef(N['Mar'], T.loc[2008:2015,'Mar'])

Если мы сравнимаем два ряда данных, то получаем четыре коэффициента, два из которых 1.0 (сам с собой всегда 1) и два - корреляция между рядами.
В данном случае это заметная отрицательная корреляция.

Из рисунка понятно, что если мы ограничим диапазон лет после 2010 года, то отрицательная корреляция будет еще сильнее.

In [ ]:
corrcoef(N.loc[2011:2015, 'Mar'], T.loc[2011:2015,'Mar'])[0,1]

В 2012 году тоже был теплый март, но перед ним был холодный февраль. Можно усреднить данные за несколько месяцев.

In [ ]:
iiyear = slice(2011,2015)
iimo = ['Jan','Feb','Mar','Apr','May']

corrcoef(N.loc[iiyear, iimo[:-1]].mean(axis=1), T.loc[iiyear, iimo[1:]].mean(axis=1))[0,1]

Итого мы обнаружили сильную отрицательную корреляцию между температурой в начале весны и масштабом весенней вспышки ротавирусной инфекции среди детей.

Корреляция может быть между любыми сходными по размеру данными.

Например, корреляция изменений температуры в марте с другими месяцами с начала 21-века.

Для таблиц `pandas` есть удобный метод для этого.

In [ ]:
C = T.loc[2000:2018].corr()
C

Чтобы единицы по диагонали не мешали восприятию рисунка, их и половину значений маскируют.

In [ ]:
mask = np.triu(np.ones_like(C))
mask

In [ ]:
sns.heatmap(C, cmap='RdBu_r', center=0, square=True, mask=mask);

На пересечении месяцев между друг другом выделяются противоположности Июль и Февраль, а также Октябрь и Август. Сходную динамику (высокая полжительная корреляция) демонстрируют Февраль-Март и Ноябрь-Декабрь. И, внезапно также Май и Январь.

Тот же прием для данных по годовой динамике заболеваемости ротавирусной инфекцией.

In [ ]:
C = N.corr()
sns.heatmap(C, cmap='RdBu_r', center=0, square=True, mask=mask);

### Фазовые сдвиги

Корреляцию можно считать между любыми рядами данных. 

В колонках таблицы рассчитаем помесячную динамику. Исключим редкие болезни с невыраженной динамикой. Чтобы подчеркнуть циклические колебания также уберем глобальный тренд.

После вычитания среднего примерно половина значений становится отрицательными. Т.е. значения - это не количество, а отклонение количества от среднего тренда.

In [ ]:
N = D.groupby(['YM','Species']).n.count().unstack()
N = N.loc[:,N.sum()>1000].fillna(0)
N[:] = detrend(N.values, 'linear', axis=0)
N.index =pd.to_datetime(N.index).to_period('M')
ax = N.plot(legend=False, figsize=(10,5));
legend(bbox_to_anchor=(1.01, 1));

In [ ]:
C = N.corr()
mask = np.triu(np.ones_like(C))

figure(figsize=(8,8))
sns.heatmap(C, cmap='RdBu_r', center=0, square=True, mask=mask);

Однако, временные ряды данных можно коррелировать не только с паралелльно изменяющимися во времени, но с самими собой, сдвинутыми по времени. Величина сдвига в шагах периода регистрации сигнала называется лаг.

Если мы сдвигаем ряд с циклической динамикой на несколько шагов, то корреляция падает, затем, когда мы доходим до противофазы (попадаем пиком туда, где ямка), то корреляция становится отрицательной, затем опять повышается. Первый большой пик автокорреляционной функции соответствует периоду основных колебаний.

In [ ]:
# pd.tools.plotting.autocorrelation_plot(N['Hepatitis C virus']); # old version
pd.plotting.autocorrelation_plot(N['Hepatitis C virus']);

In [ ]:
pd.plotting.autocorrelation_plot(N['Parainfluenza']);

In [ ]:
pd.plotting.autocorrelation_plot(N['Rotavirus']);

Для рассчета исходной корреляционной функции без построения рисунка используют соответствующую функцию.

In [ ]:
vv = N['Rotavirus']
c = correlate(vv, vv, 'full')
plot(c);
vv.shape, c.shape

Поскольку мы можем двигать один ряд значений относительно другого и вперед и назад, то полная функция имеет удвоенную длину. На предыдущем рисунке была показана только ее правая половина, причем нормализованная к общей мощности сигнала, чтобы не выходить за рамки диапазона от -1 до 1.

Давайте получим нормализованные значения корреляции.

In [ ]:
c = c[len(c)//2:]
c = c / (vv.var()) / len(c)
plot(c); grid(True);
c.shape

Вместо того, чтобы делить на дисперсию исходного сигнала, правильнее нормализовать входные ряды данных. Мы уже вычли тренд из динамики. Давайте теперь поделим каждую колонку на стандартное отклонение ее значений.

In [ ]:
N = N / N.std()
ax = N.plot(legend=False, figsize=(10,5));
legend(bbox_to_anchor=(1.01, 1));

Резко преобладающие по численности пики RSV теперь незаметны на фоне остальных.

Годовые циклы имеют лаг в 12 месяцев. Однако какой лаг между динамикой разных заболеваний?

In [ ]:
vv1 = N['Rotavirus']
vv2 = N['Respiratory Syncytial Virus']
c = correlate(vv1, vv2, 'full')
c = c[len(c)//2:]
c = c/len(c)
plot(c);

In [ ]:
c.max(), c.argmax()

Между рядами высокая корреляция - 0.62, но сезонный пик в первом ряду значений сдвинут на 4 месяца относительно сезонного пика во втором ряду значений.

In [ ]:
vv1.plot()
vv2.plot()
legend();

### Скачкообразная динамика

Некоторые болезни "привязаны" к некоторым странам, потому что там живут специфические переносчики каких-то из жизненных стадий паразитов.

Например, вирус chikungunya из Африки передается при укусах комаров, которые в Бельгии жить не должны. Это тропическая лихорадка. В Бельгии очень мало случаев. Посмотрим помесячную динамику. 

Сделаем помесячный индекс, охватывающий все данные в наборе.

In [ ]:
# monthindex = pd.to_datetime(D.DateMonday.unique()).to_period('M').unique()
monthindex = pd.PeriodIndex(start='2008-01', end='2015-12')
monthindex

In [ ]:
X = D[D.Subject=='CHIK']
N = X.groupby(['YM']).M.count()
N

Теперь сделаем пустую серию с полным индексом и заполним те значения, которые мы посчитали.

In [ ]:
S = pd.Series(0, index=monthindex)

ii = pd.to_datetime(N.index).to_period('m')
ii
S[ii] = N

S.plot();

Видно, что случаи болезни были единичными - возможно болезнь привозили туристы из поездок в теплые страны. Но с 2014 количество заболевших этой тропической лихорадкой резко возрасло.

И этот график идеально совпадает с [волной мигрантов, захлестнувшей Европу в 2014-2015 гг](https://en.wikipedia.org/wiki/European_migrant_crisis#Global_refugee_crisis).
Среди этих мигрантов и были носители вируса Чикунгунья.

### Эффекты развития здравоохранения

На таком коротком временном отрезке (8 лет) трудно говорить о влиянии развития медицины, хотя безусловно оно есть.

В данном наборе есть один пример резкого снижения заболеваемости, который может быть объяснен успешными противоэпидемиологическими мерами.

Для демонстрации сделаем сводную таблицу динамики по годам, причем нормализуем ее, отняв среднее и разделив на стандартное отклонение.

In [ ]:
N = D.pivot_table('n', 'Y', 'Species', 'sum').fillna(0)
N = (N - N.mean())/ N.std()
figure(figsize=(8,6))
sns.heatmap(N.T, cmap='coolwarm', center=0);

Сильный "скачок" вниз наблюдался для заболеваний, вызванных Entamoeba histolytica и ВИЧ. 
После 2008 года только снижалась заболеваемость сальмонеллезом и гепатитом А.
Как мы знаем значимыми изменениями часто считают те, что выходят за 2 "сигмы". Поскольку те значения, что мы получили после нормализации, как раз в сигмах, то для наглядности мы можем скрыть значения, не достигшие "уровня значимости".

In [ ]:
@interact(porog=(0.5, 2.5))
def _fig(porog=1.96):
    #figure(figsize=(8,6))
    sns.heatmap(N.T, cmap='RdBu_r', center=0, mask=N.T.abs()<porog);

Другой подход оценить отклонения - зафиксировать исходный нулевой уровень на базе начальных данных, например, до 2012 года. Тогда изменения от этого уровня в левой части рисунка будут нивелированы, в правой будут отражать относительные сдвиги в последующие годы.

In [ ]:
N_ = N - N[N.index < 2013].mean()
figure(figsize=(8,6))
sns.heatmap(N_.T, cmap='RdBu_r', center=0);
title('Все сдвиги')
figure(figsize=(8,6))
sns.heatmap(N_.T, cmap='RdBu_r', center=0, mask=N_.T>0);
title('Только снижение');

Чтобы оценить уровень развития здравоохранения в Бельгии, можно сопоставить графики увеличения продолжительности жизни за последние полвека (Trajectories through inequality/life expectancy space). Черные точечки - данные по отдельным годам для одной страны. Зеленое облако - данные за всю историю наблюдений по всем странам.

Бельгии | Россия
-|-
![](https://globaldatalab.org/assets/2017/01/LLI_Belgium.jpg) | ![](https://globaldatalab.org/assets/2017/01/LLI_Russia.jpg)

## Примеры решения задач

### Задача на прогноз по предыдущим значениям

По данным epistat постройте прогноз динамики распространения хламидиоза
на неделю, начинающуюся c 2014-09-01, на основании предыдущих данных, 
используя метод скользящего среднего с окном в 4 нед.
Ошибка в рассчетах составит по сравнению с реальным уровнем?

In [ ]:
X = D[D.Subject=='CHLTRA']
N = X.groupby('DateMonday').DateMonday.count()
N

nreal = N['2014-09-01']
nreal

In [ ]:
nnprev = N[ (N.index < '2014-09-01') & (N.index > '2014-07-01') ]
nnprev

In [ ]:
nprognoz = round(nnprev.iloc[-4:].mean())

nerror = nprognoz - nreal

nprognoz, nreal, nerror

### Задача на линейную экстраполяцию

```
Вы моделируете динамику заболеваемости по годам с помощью экстраполяции
линейной регрессионной модели за предыдущие 8 лет.
В каком регионе Бельгии прогнозируется самый сильный относительный рост
заболеваемости респираторно-синцитиальной инфекции в 2016 году (по отношению к 2015 году)?
Выберите один ответ:
a. Hainaut
b. Walloon Brabant
c. West Flanders
d. Antwerp
```

In [ ]:
ny = 8
X = D[D.Subject=='V_RSV']
N = X.pivot_table('W', 'Y', 'Province', 'count').fillna(0)
N

In [ ]:
#берем последние 8 лет
N = N.iloc[-ny:,:]

Потренируемся на одной колонке.

In [ ]:
y = N.iloc[:,4]
x = arange(len(y))
plot(x,y,'o');

# добавим тренд
res = stats.linregress(arange(len(y)), y)
print(res)
xx = arange(ny+1)
yy = res.intercept + res.slope*xx
plot(xx,yy, '-r');

prognoz = yy[-1]
plot(xx[-1], yy[-1], 'ro');

In [ ]:
def _linreg(y):
    res = stats.linregress(arange(len(y)), y)
    return pd.Series(res, index=['slope', 'intercept', 'r', 'p', 'stderr'])

R = N.apply(_linreg, axis=0).T
R

In [ ]:
prognoz = (R.intercept + R.slope*(ny)).astype(int)
prognoz = prognoz.clip_lower(0)
prognoz

In [ ]:
last = N.iloc[-1,:]
rost = (prognoz - last) / last
rost

In [ ]:
rost.idxmax()

In [ ]:
pname = 'Walloon Brabant'
plot(N.index, N[pname], 'bo')
plot(2016, prognoz[pname], 'ro');

В 2015 было всего 42 случая, но экстраполяция снижающегося тренда дала на 2016 год значение 109, что больше примерно в полтора раза.

### Задача на лаг в неделях


Каков лаг в неделях между сезонными колебаниями заболеваемости шигеллёза и аденовирусной инфекции ?


In [ ]:
X = D[D.Subject.isin(['SHIG','V_ADV'])]
N = X.groupby(['W','Subject']).n.sum().unstack().fillna(0)
N[:] = detrend(N.values, 'linear', axis=0)
N = N / N.std()
N.plot();

In [ ]:
a = N.iloc[:,0]
b = N.iloc[:,1]
    
c = np.correlate(a, b, 'full')
c = c[len(c)//2:]
c = c/len(c)
    
plot(c);

In [ ]:
365/7

Более подробно кросс-корелляционная функция на год (~52 недели).

In [ ]:
plot(c);
xlabel('Лаг, нед')
ylabel('Корреляция')
xlim(0, 52);

In [ ]:
c.argmax()

Тридцать недель - это примерно 6 месяцев, т.е. простудное заболевание и кишечное отравление имеют пики зимой и летом.